# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from scipy.optimize import *
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *
from stlab.devices.RS_SGS100A import RS_SGS100A
from stlab.devices.Keysight_N5183B import Keysight_N5183B

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice import config, RR_1_IF,RO_lo, readout_len, Q1_lo, Q1_IF, sat_Q1_len, gauss_len, R1_RS, Q1_RS
print(readout_len)

# Edit configuration parameters

**To do**

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.184.193::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(-15)
RR.setCWfrequency(RO_lo)
RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
RR.IQon()
RR.write(':SOURce:IQ:IMPairment:STATe ON')

#LO for downconversion
MXG = Keysight_N5183B(addr='TCPIP::192.168.1.91::INSTR',reset=True,verb=True)
MXG.RFon()
MXG.setCWpower(19)
MXG.setCWfrequency(RO_lo)
MXG.INTref()

QDrive = RS_SGS100A("TCPIP::169.254.50.124::INSTR", reset=True,verb=True) 
QDrive.EXTref()
QDrive.RFon()
QDrive.setCWpower(-35)
QDrive.setCWfrequency(Q1_lo)
QDrive.write(':SOURce:IQ:IMPairment:LEAKage:I ' + Q1_RS[0])
QDrive.write('SOURce:IQ:IMPairment:LEAKage:Q ' + Q1_RS[1])
QDrive.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + Q1_RS[2])
QDrive.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + Q1_RS[3])
QDrive.IQon()
QDrive.write(':SOURce:IQ:IMPairment:STATe ON')

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# QUA Program

In [ ]:
def Time_Rabi(n_max, f, a, b, t_vec, QLO_Power): 
    QDrive.setCWpower(QLO_Power)
    
    t_min = int(np.min(t_vec))
    t_max = int(np.max(t_vec))
    dt = int(t_vec[1] - t_vec[0])

    with program() as time_Rabi:
        ##############################
        # declare real-time variables:
        ##############################
        
        n = declare(int)        # Averaging
        tau = declare(int)

        A = declare(fixed)
        B = declare(fixed)
        iA = declare(fixed)
        iB = declare(fixed)
        Re = declare(fixed)
        Im = declare(fixed)

        Re_st = declare_stream()
        Im_st = declare_stream()

        ###############
        # the sequence:
        ###############
        
        update_frequency("Q1", f)
        
        with for_(n, 0, n < n_max, n + 1):
            with for_(tau, t_min, tau <= t_max, tau+dt):    
                wait(int(5000 / 4), "RR_1", "Q1")
                play("gaussian" * amp(a), "Q1", duration=tau)  
                align("Q1", "RR_1")
                measure("readout" * amp(b), "RR_1", None, demod.full("integW_cos", A, "out1"),
                        demod.full("integW_sin", B, "out2"),
                        demod.full("integW_sin", iA, "out1"),
                        demod.full("integW_cos", iB, "out2"))

                assign(Re, A + B)
                assign(Im, iA - iB)
                save(Re, Re_st)
                save(Im, Im_st)

        with stream_processing():
            Re_st.buffer(len(t_vec)).average().save("Re")
            Im_st.buffer(len(t_vec)).average().save("Im")
            
    job = qm.execute(time_Rabi, duration_limit=0, data_limit=0)

    res_handles= job.result_handles
    res_handles.wait_for_all_values()

    Re_handle = res_handles.get("Re")
    Im_handle = res_handles.get("Im")
   
    Re_s = Re_handle.fetch_all()
    Im_s = Im_handle.fetch_all()
   
    return Re_s, Im_s 

# Define Parameters

In [ ]:
n_max = 1e6

#Readout pulse amplitude
b = 0.24

# Qubit pulse amp 
a = 1 #amp_Q1_pi

#Qubit pulse time (in clock cycles of 4ns)
t_min = 6 #5
t_max = 125
dt = 1
t_vec = np.arange(t_min,t_max+dt/2,dt)

#Qubit pulse frequency
f_q = 5.86544e9
qd_IF = int(f_q - Q1_lo)

#Qubit Pulse Power
QLO_Power = 0

print(qd_IF/1e6)
print(len(t_vec))
print(t_vec)

In [ ]:
Re, Im = Time_Rabi(n_max, qd_IF, a, b, t_vec, QLO_Power)

## Fetch data and save to .dat

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'Q1_Time_Rabi'

f_min = qd_IF
f_max = qd_IF
df = 10e6
f_vec = np.arange(f_min, f_max+df/2, df)

data_Re = Re.reshape(len(Re),1)
data_Im = Im.reshape(len(Im),1)
data_Sig = (np.abs(data_Re + 1j*data_Im)).reshape(len(Re),1)
data_Amp = (20*(np.log10(np.abs(data_Sig)))).reshape(len(Re),1)
data_Ph = (np.unwrap(np.arctan(data_Im/data_Re))).reshape(len(Re),1)

data = np.asarray([(Q1_lo+(f_vec.reshape(len(f_vec),1)+t_vec)-t_vec).T,((t_vec.reshape(len(t_vec),1)+f_vec)-f_vec),data_Re,data_Im,data_Sig,data_Amp,data_Ph])
for i,ts in enumerate(t_vec):
    print(i,ts)
    data_dict={'Qubit Frequency (Hz)':data[0][i],
           'Qubit Pulse Time (ns)':data[1][i]*4,
           'Real':data[2][i],
           'Imaginary':data[3][i],
           'Signal':data[4][i],
           'Amplitude (dB)':data[5][i],
           'Phase (rad)':data[6][i]
    }
    if i==0:
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        
        myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True, git_id = False)
        
        output = new_stdout.getvalue()
        sys.stdout = old_stdout
        print(output)
        M_ind = output.find("Measurement Name")
        M_name = output[M_ind+len('Measurement Name:  '):-1]
    stlab.savedict(myfile,data_dict)
    stlab.metagen.fromarrays(myfile,data_dict['Qubit Frequency (Hz)'], t_vec[0:i+1]*4,xtitle='Qubit Frequency (Hz)',ytitle='Qubit Pulse Time (ns)',colnames=list(data_dict))

# Processing/Plotting

In [ ]:
sig = Re + 1j*Im

abs_sig = np.abs(sig)
power = 20*(np.log10(np.abs(sig)))#+10*np.log10(1000/50)#np.abs(sig)   #equation from wiki
power2=power/np.mean(power,axis=0)                             #norm lbl
power3=(scipy.ndimage.gaussian_filter(1/(power/np.mean(power,axis=0)),[5])) #gaussian filtered norm lbl
power4=power-np.mean(power,axis=0)                                    #sub lbl
power5=scipy.ndimage.gaussian_filter(power,[3])-np.mean(power,axis=0)#gaussian filtered sub lbl
power6=scipy.ndimage.gaussian_filter(power,[3]) #just gaussian filtered
phase = np.unwrap(np.arctan(Im/Re))

from scipy.optimize import curve_fit

def f_cos(t,A,drive_period,phi,gamma,C):
    return A*np.cos(2 * np.pi * t / drive_period + phi)*np.exp(-t*2*np.pi/gamma) + C

popt, pcov = curve_fit(f_cos, t_vec, abs_sig,p0 = [2.2e-5, 90, np.pi/4,800,4e-5])


plt.figure(num=None, figsize=(8, 6), dpi=100)
plt.tight_layout()

plt.rcParams['axes.titlepad'] = 14
plt.title('Time Rabi, Q_amp={}, t_RO={}ns, R_amp = {}, Qfreq  = {} GHz, QLO_Power = {} dBm, n = {}'.format(a,readout_len,b, f_q/1e9,QLO_Power, n_max)) 
plt.plot(t_vec*4,abs_sig)

t = np.linspace(0,t_vec[-1],500)
plt.plot(t*4,f_cos(t,popt[0],popt[1],popt[2],popt[3],popt[4]),label = 'fit')

plt.xlabel('Qubit Pulse Time (ns)', fontsize = 16)
plt.ylabel('|I+iQ| (a.u.)', fontsize = 16)
plt.xlim(0,t[-1]*4)
plt.xticks(fontsize = 16); plt.yticks(fontsize = 16)
plt.legend(fontsize = 14)
print('The pi pulse time is', np.round(popt[1]*4/2,3), 'ns.')

print(np.sqrt(np.diag(pcov))[1]*4)

# Save this file and configuration file to measurement folder

In [ ]:
#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_BMDevice.py')
save_config_path = os.path.join(meas_path, 'Configuration_BMDevice.py')

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);